In [1]:
import pandas as pd  #importing all the important packages
import numpy as np
import matplotlib.pyplot as plt #currently not used
import seaborn as sns #currently not used
import pickle
pickle_in = open('uptogen8pokemon.pickle','rb')
df = pickle.load(pickle_in)
df.columns = df.columns.str.upper().str.replace('_', '') 
df['TYPE2'] = df['TYPE2'].replace(to_replace=[None], value=np.nan, inplace=True)
df['TYPE2'].fillna(df['TYPE1'], inplace=True)
df = df.set_index('NAME')
df=df.drop(['ID'],axis=1)

# Damage multiplier for type matchups

The function accepts two types, the first type is the attacking move type and the second is the defenders type (if the defender has multiple types this function should be run multiple times and multiplied together).

In [2]:
def type_weakness(type1,type2):
    if type1 == 'normal':
        if type2 in ['rock' ,'steel']:
            return 0.5
        elif type2 == 'ghost':
            return 0
    if type1 == 'fire':
        if type2 in ['water','fire','rock', 'dragon']:
            return 0.5
        elif type2 in ['grass', 'ice', 'bug','steel']:
            return 2
    if type1 == 'water':
        if type2 in ['water', 'grass', 'dragon']:
            return 0.5
        if type2 in ['fire', 'ground', 'rock']:
            return 2
    if type1 == 'grass':
        if type2 in ['fire', 'grass', 'poison' , 'dragon', 'steel', 'bug', 'flying']:
            return 0.5
        if type2 in ['water' , 'ground' ,'rock']:
            return 2
    if type1 == 'electric':
        if type2 == 'ground':
            return 0
        if type2 in ['grass', 'electric', 'dragon']:
            return 0.5
        if type2 in ['water','flying']:
            return 2
    if type1 == 'ice':
        if type2 in ['fire', 'water', 'ice', 'steel']:
            return 0.5
        if type2 in ['grass','flying','ground', 'dragon']:
            return 2
    if type1 == 'fighting':
        if type2 == 'ghost':
            return 0
        if type2 in ['poison', 'flying', 'psychic', 'bug', 'fairy']:
            return 0.5
        if type2 in ['normal', 'ice','rock', 'dark', 'steel']:
            return 2
    if type1 == 'poison':
        if type2 == 'steel':
            return 0
        if type2 in ['poison', 'ground', 'rock', 'ghost']:
            return 0.5
        if type2 in['fairy', 'grass']:
            return 2
    if type1 == 'ground':
        if type2 == 'flying':
            return 0
        if type2 in['grass','bug']:
            return 0.5
        if type2 in ['fire', 'electric','rock', 'steel', 'poison']:
            return 2
    if type1 == 'flying':
        if type2 in['electric','steel', 'rock']:
            return 0.5
        if type2 in ['grass', 'fighting', 'bug']:
            return 2
    if type1 == 'psychic':
        if type2 == 'dark':
            return 0
        if type2 in['steel','psychic']:
            return 0.5
        if type2 in ['fighting', 'poison']:
            return 2
    if type1 == 'bug':
        if type2 in ['fire', 'fighting', 'poison', 'flying', 'ghost' ,'steel', 'fairy']:
            return 0.5
        if type2 in ['dark','psychic','ghost']:
            return 2
    if type1 == 'rock':
        if type2 in ['steel', 'fighting', 'ground']:
            return 0.5
        if type2 in ['bug' ,'flying','ice','fire']:
            return 2
    if type1 == 'ghost':
        if type2 == 'normal':
            return 0
        if type2 == 'dark':
            return 0.5
        if type2 in ['psychic','ghost']:
            return 2
    if type1 == 'dragon':
        if type2 == 'fairy':
            return 0
        if type2 == 'steel':
            return 0.5
        if type2 == 'dragon':
            return 2
    if type1 == 'dark':
        if type2 in ['fighting','dark','fairy']:
            return 0.5
        if type2 in ['ghost','psychic']:
            return 2
    if type1 == 'steel':
        if type2 in ['steel','fire','water','electric']:
            return 0.5
        if type2 in ['ice','rock','fairy']:
            return 2
    if type1 == 'fairy':
        if type2 in ['fire', 'poison', 'steel']:
            return 0.5
        if type2 in ['dragon', 'fighting', 'dark']:
            return 2
    return 1

# Damage Calculations

In [3]:
def gen1_dmg(attack, defence, lvl, crit, power, movetype, physical,stab,deftype1,deftype2):
    #initial dmg
    dmg = 2*lvl
    #if crits
    if crit == True:
        dmg = 2*dmg
    #extras
    dmg = dmg/5 + 2
    #move power
    dmg = dmg*power
    #Physical or Special dmg
    dmg = dmg*attack/defence
    #extras
    dmg =dmg/50+2
    #STAB dmg
    if stab == True:
        dmg = dmg*1.5
    #consider weaknesses
    if deftype1 == deftype2:
        dmg = dmg*type_weakness(movetype,deftype1)
    else:
        dmg = dmg*type_weakness(movetype,deftype1)*type_weakness(movetype,deftype2)
    return [int(dmg*(217/255)),int(dmg)]

In [4]:
def gen2_dmg(attack, defence, lvl, crit, power, movetype, physical,stab,deftype1,deftype2,item,badge,weather):
    #initial dmg
    dmg = 2*lvl/5+2
    #move power
    dmg = dmg*power
    #Physical or Special dmg
    dmg = dmg*attack/defence
    #extras
    dmg =dmg/50
    #if crits
    if crit == True:
        dmg = 2*dmg
    #item multiplier
    dmg = dmg*item
    #extras
    dmg = dmg+2
    #STAB dmg
    if stab == True:
        dmg = dmg*1.5
    #badge
    dmg = dmg*badge
    #weather
    if weather == True:
        dmg = dmg*1.5
    #consider weaknesses
    if deftype1 == deftype2:
        dmg = dmg*type_weakness(movetype,deftype1)
    else:
        dmg = dmg*type_weakness(movetype,deftype1)*type_weakness(movetype,deftype2)
    
    return [int(dmg*(217/255)),int(dmg)]

In [5]:
def gen3_dmg(attack, defence, lvl, crit, power, movetype, physical,stab,deftype1,deftype2,item,weather,burn,screen,double,targetall,FF,WBR,HH,charge):
    #initial dmg
    dmg = 2*lvl/5+2
    #move power
    dmg = dmg*power
    #Physical or Special dmg
    dmg = dmg*attack/defence
    #extras
    dmg =dmg/50
    #burned (and not guts)
    if burn == True:
        dmg = dmg/2
    #screen
    if screen == True:
        if double == True:
            dmg = dmg*2/3
        else:
            dmg = dmg/2
    #in double battle dmg split between targets
    if double == True and targetall == False:
        dmg = dmg/2
    #flash fire
    if FF == True:
        dmg = dmg*1.5
    #weather
    if weather == True:
        dmg = dmg*1.5
    #extras
    dmg = dmg+2
    #item multiplier (assuming this goes here)
    dmg = dmg*item
    #extras
    dmg = dmg+2
    #STAB dmg
    if stab == True:
        dmg = dmg*1.5
    #consider weaknesses
    if deftype1 == deftype2:
        dmg = dmg*type_weakness(movetype,deftype1)
    else:
        dmg = dmg*type_weakness(movetype,deftype1)*type_weakness(movetype,deftype2)
    #if crits
    if crit == True:
        dmg = 2*dmg
    #weatherball revenge (only in clear weather)
    if WBR == True:
        dmg = dmg*2
    #helping hand
    if HH == True:
        dmg = dmg*1.5
    #electric type charge effect
    if charge == True:
        dmg =dmg*2
    return [int(dmg*(85/100)),int(dmg)]

In [6]:
def gen4_dmg(attack, defence, lvl, crit, power, movetype, physical,stab,deftype1,deftype2,item,weather,burn,screen,double,targetall,FF,WBR,HH,charge,other):
    #initial dmg
    dmg = 2*lvl/5+2
    #move power
    dmg = dmg*power
    #Physical or Special dmg
    dmg = dmg*attack/defence
    #extras
    dmg =dmg/50
    #burned (and not guts)
    if burn == True:
        dmg = dmg/2
    #screen
    if screen == True:
        if double == True:
            dmg = dmg*2/3
        else:
            dmg = dmg/2
    #in double battle dmg split between targets
    if double == True and targetall == False:
        dmg = dmg/2
    #flash fire
    if FF == True:
        dmg = dmg*1.5
    #weather
    if weather == True:
        dmg = dmg*1.5
    #extras
    dmg = dmg+2
    #item multiplier (assuming this goes here)
    dmg = dmg*item
    #extras
    dmg = dmg+2
    #STAB dmg
    if stab == True:
        dmg = dmg*1.5
    #consider weaknesses
    if deftype1 == deftype2:
        dmg = dmg*type_weakness(movetype,deftype1)
    else:
        dmg = dmg*type_weakness(movetype,deftype1)*type_weakness(movetype,deftype2)
    #if crits
    if crit == True:
        dmg = 2*dmg
    #weatherball revenge (only in clear weather)
    if WBR == True:
        dmg = dmg*2
    #helping hand
    if HH == True:
        dmg = dmg*1.5
    #electric type charge effect
    if charge == True:
        dmg =dmg*2
    #any other abilities and effects
    dmg = dmg*other
    return [int(dmg*(85/100)),int(dmg)]

In [7]:
def gen5_up_dmg(attack, defence, lvl, crit, power, movetype, physical,stab,deftype1,deftype2,item,weather,burn,screen,double,targetall,FF,WBR,HH,charge,other):
    #initial dmg
    dmg = 2*lvl/5+2
    #move power
    dmg = dmg*power
    #Physical or Special dmg
    dmg = dmg*attack/defence
    #extras
    dmg =dmg/50+2
    #burned (and not guts)
    if burn == True:
        dmg = dmg/2
    #screen
    if screen == True:
        if double == True:
            dmg = dmg*2/3
        else:
            dmg = dmg/2
    #in double battle dmg split between targets
    if double == True and targetall == False:
        dmg = dmg/2
    #flash fire
    if FF == True:
        dmg = dmg*1.5
    #weather
    if weather == True:
        dmg = dmg*1.5
    #item multiplier (assuming this goes here)
    dmg = dmg*item
    #extras
    dmg = dmg+2
    #STAB dmg
    if stab == True:
        dmg = dmg*1.5
    #consider weaknesses
    if deftype1 == deftype2:
        dmg = dmg*type_weakness(movetype,deftype1)
    else:
        dmg = dmg*type_weakness(movetype,deftype1)
        dmg = dmg*type_weakness(movetype,deftype2)
    #if crits
    if crit == True:
        dmg = 2*dmg
    #weatherball revenge (only in clear weather)
    if WBR == True:
        dmg = dmg*2
    #helping hand
    if HH == True:
        dmg = dmg*1.5
    #electric type charge effect
    if charge == True:
        dmg =dmg*2
    #any other effects; e.g. abilities
    dmg = dmg*other
    return [int(dmg*(85/100)),int(dmg)]

# Base Stats at Level

We use the following function to determine a pokemons base stats at a certain level, this does not include IVs, EVs or natures. However, in the function IVs and EVs can be seen. Natures would multiply one stat by 1.1 and another by 0.9.

In [8]:
def base_stats_at_lvl(Pokemon,level):
    pkm = df[df.index == Pokemon]
    IV = 0#[0,0,0,0,0,0]
    EV = 0#[0,0,0,0,0,0]
    HP = ((2*pkm['HP']+IV+(EV/4))*level)/100+level+10
    ATK = ((2*pkm['ATK']+IV+(EV/4))*level)/100+5
    DEF = ((2*pkm['DEF']+IV+(EV/4))*level)/100+5
    SPATK = ((2*pkm['SPATK']+IV+(EV/4))*level)/100+5
    SPDEF = ((2*pkm['SPDEF']+IV+(EV/4))*level)/100+5
    SPD = ((2*pkm['SPD']+IV+(EV/4))*level)/100+5
    return [int(HP),int(ATK),int(DEF),int(SPATK),int(SPDEF),int(SPD)]

# A/D Ratio

We create a function to calculate the A/D ratio between two pokemon. This is the difference in the attack stat with the defence stat or the special attack stat with the special defence stat. (There are some exceptions such as Psyshock and Psystrike).

In [9]:
def phys_spec_ratio(atk_pkm,atk_lvl,def_pkm,def_lvl):
    pkm1 = base_stats_at_lvl(atk_pkm,atk_lvl)
    pkm2 = base_stats_at_lvl(def_pkm,def_lvl)
    phys = pkm1[1]/pkm2[2]
    spec = pkm1[3]/pkm2[4]
    return[float(phys),float(spec)]

# Function Testing

In [10]:
gen1_dmg(attack=27,defence=16,lvl=15,crit=True,power=40,movetype='normal',physical=True,stab=True,deftype1='normal',deftype2='normal')


[26, 31]

In [11]:
gen5_up_dmg(attack=27,defence=16,lvl=15,crit=True,power=40,movetype='fairy',physical=True,stab=False,deftype1='dark',deftype2='dragon',item=1,weather=False,burn=False,screen=False,double=False,targetall=False,FF=False,WBR=False,HH=False,charge=False,other=1)

[100, 118]

In [12]:
type_weakness('fairy','dark')

2

In [13]:
base_stats_at_lvl('Patrat',15)

[38, 21, 16, 15, 16, 17]

In [14]:
phys_spec_ratio('Patrat',15,'Snubull',15)

TypeError: cannot convert the series to <class 'int'>